# Project #2: Automation of Concrete Mix Design (NDOT)
**Team:** Group 6  
**Date:** <2026-02-24>  
**Client:** Nebraska Department of Transportation (NDOT)

---

## Project Context (Client Summary)
NDOT has requested a Python-based automation of its Excel “Mix Design” worksheet logic. The workflow must:
1) translate Excel logic into Python functions,  
2) prompt the user for inputs sequentially (mirroring Excel order),  
3) generate a client-readable weight chart for **one cubic yard**, and  
4) evaluate **four realistic mix scenarios** as basic verification/validation. 
---

## Deliverables Checklist 
- [ ] Excel logic replicated using **major calculation functions**  
- [ ] **Sequential user prompts** implemented 
- [ ] **Final weight chart** for 1 yd³ generated and formatted  
- [ ] **4 mix scenarios** documented + run through the model  
- [ ] Scenario outputs compared (quick verification/validation narrative)

---

# Part 1 - Setup

## 1.1 Imports and Global Settings

- Units convention:
  - Cement, water, aggregates: **lb/yd³**
  - Specific gravity: dimensionless
  - Air content: %
  - w/c ratio: dimensionless

---

## 1.2 Assumptions and Constants
▶️ **Code Cell:** define constants (e.g., density of water, unit conversions, etc.)

Document assumptions here:
- using imperial units


---

In [3]:
#Part 1 Code
#Unit conversions, contstants, and packages
import numpy as np
import pandas as pd

cubic_yd_ft3 = 27 #cubic feet in 1 CY
unit_weight_water = 62.4 #lb/ft^3

## 1.3 Inputs Dictionary 
▶️ **Code Cell:** create a dictionary or dataclass for all inputs

Recommended structure:
- cement_A = "what does this variable mean"
- any additional parameters from the Excel logic you are replicating
---

# Part 2 - Cement Calculation Function


## 1. Water Weight (lb/yd³)

In the NDOT Mix Design worksheet, water weight is computed as: 

water weight = (weight of cement + weight of fly ash + weight of silica fume + weight of other SCM) × target water-cement ratio


**Inputs:**
- cement content
- w/c ratio

**Output:**
- water weight (lb/yd³)

---


In [4]:
def water_weight(cement_a, fly_ash_b, silica_fume_c, other_scm_d, wc_ratio_e):
    total_cementitous_weight = cement_a + fly_ash_b + silica_fume_c + other_scm_d
    water_weight_Q = total_cementitous_weight * wc_ratio_e
    return water_weight_Q

## 2. Material Volumes Using Specific Gravity

Each material volume is calculated using the same general relationship:

Volume = Weight / (Specific Gravity × Unit Weight of Water)

**Inputs:**
- weight of material
- sg of material

**Output:**
- volume of material (ft^3)

---


In [5]:
#Functions
#For cement volume:
def vol_cement(w_cement, sg_cement):
    vol_R = w_cement / (sg_cement * unit_weight_water)
    return vol_R

#For fly ash volume
def fly_ash_vol(w_flyash, sg_flyash):
    vol = w_flyash /(sg_flyash * unit_weight_water)
    return vol

#For silica fume volume
def silica_vol(w_silica, sg_silica):
    vol = w_silica /(sg_silica * unit_weight_water)
    return vol

#For other SCM volume:
def scm_vol(w_scm, sg_scm):
    vol = w_scm /(sg_scm * unit_weight_water)
    return vol

#For air volume:
def air_vol(target_air_content):
    return (target_air_content / 100) * cubic_yd_ft3

#For water volume
def water_vol(weight_water):
    return weight_water / unit_weight_water


## 3. Calculating Total Aggregate Volume

At this stage in the NDOT Mix Design process, all non-aggregate volumes have been accounted for. The remaining volume must be filled by aggregates.

Total aggregate Volume = total volume - cementituous material volume - air volume - water volume.

**Inputs:**
- total volume
- cementituous material volume
- air volume
- water volume

**Output:**
- total aggregate volume

---


In [6]:
#Function to calculate aggregate volume
def agg_vol(R, S, T, U, V, W):
    return 27 - R - S - T - U - V - W

## 4. Calculating Fine and Coarse Aggregate Weight

Once the total aggregate volume (X) has been determined, it must be divided among individual aggregate types based on the target aggregate percentages.

The formula for fine aggregate weight is:
fine aggregate weight = (unit weight of water lb/ft^3)* (target % fine aggregate / 100) * specific gravity of fine aggregate * total aggregate volume

The formula for coarse aggregate weight is:
coarse aggregate weight = (unit weight of water lb/ft^3)* (target % coarse aggregate / 100) * specific gravity of coarse aggregate * total aggregate volume

The other aggregate follows the same structure and logic.


For the fine aggregate weight function:

**Inputs:**
- target percent fine aggregate (%)
- specific gravity of fine aggregate
- total aggregate volume (ft³)
- unit weight of water (lb/ft³)

**Output:**
- fine aggregate weight


For the coarse aggregate weight function:

**Inputs:**
- target percent coarse aggregate (%)
- specific gravity of coarse aggregate
- total aggregate volume (ft³)
- unit weight of water (lb/ft³)

**Output:**
- coarse aggregate weight


For the other aggregate weight function:

**Inputs:**
- target percent other aggregate (%)
- specific gravity of other aggregate
- total aggregate volume (ft³)
- unit weight of water (lb/ft³)

**Output:**
- other aggregate weight
---


In [12]:
#Function for fine aggregate weight
def fine_aggregate_Y(percent_fine_G, sg_fine_N, volume_X):
    weight_Y = unit_weight_water * (percent_fine_G /100) * sg_fine_N * volume_X
    return weight_Y

#Function for coarse aggregate weight
def coarse_aggregate_Z(percent_coarse_H, sg_coarse_O, volume_X):
    return unit_weight_water * (percent_coarse_H /100) * sg_coarse_O * volume_X

#Function for other aggregate weight
def other_aggregate_AA(percent_other_I, sg_other_P, volume_X):
    return unit_weight_water * (percent_other_I /100) * sg_other_P * volume_X


# Part 3 - Sequential User Inputs 
**Enter concrete information for class of concrete below**


In [ ]:
#user inputs
#Project metadata
project_number = int(input("Enter project number: "))
class_of_concrete = input("Enter class of concrete: ")

#Cementitious material inputs (lb per cubic yard)
cement_A = float(input("Enter cement weight A (lb per cubic yard): "))
fly_ash_B = float(input("Enter fly ash weight B (lb per cubic yard): "))
silica_fume_C = float(input("Enter silica fume weight C (lb per cubic yard): "))
other_scm_D = float(input("Enter other SCM weight D (lb per cubic yard): "))

water_cement_ratio_E = float(input("Enter target water-cement ratio E: "))
air_content_F = float(input("Enter target air content F (%): "))

#Aggregate proportions (%)
percent_fine_G = float(input("Enter percent fine aggregate G (%): "))
percent_coarse_H = float(input("Enter percent coarse aggregate H (%): "))
percent_other_I = float(input("Enter percent other aggregate I (%): "))


#Specific gravities
sg_cement_J = float(input("Enter specific gravity of cement J: "))
sg_fly_ash_K = float(input("Enter specific gravity of fly ash K: "))
sg_silica_fume_L = float(input("Enter specific gravity of silica fume L: "))
sg_other_scm_M = float(input("Enter specific gravity of other SCM M: "))

sg_fine_N = float(input("Enter specific gravity of fine aggregate N: "))
sg_coarse_O = float(input("Enter specific gravity of coarse aggregate O: "))
sg_other_P = float(input("Enter specific gravity of other aggregate P: "))

# Part 4 - Running Complete Mix Design

Run your functions, output a table of the design (Lab #3 will show you how to do that)

In [ ]:
# Calculations
Q = water_weight(cement_A, fly_ash_B, silica_fume_C, other_scm_D, water_cement_ratio_E)

R = vol_cement(cement_A, sg_cement_J)
S = fly_ash_vol(fly_ash_B, sg_fly_ash_K)
T = silica_vol(silica_fume_C, sg_silica_fume_L)
U = scm_vol(other_scm_D, sg_other_scm_M)

V = air_vol(air_content_F)
W = water_vol(Q)

X = agg_vol(R, S, T, U, V, W)

Y = fine_aggregate_Y(percent_fine_G, sg_fine_N, X)
Z = coarse_aggregate_Z(percent_coarse_H, sg_coarse_O, X)
AA = other_aggregate_AA(percent_other_I, sg_other_P, X)


# Output: Final Weight Chart
print("\n---------------------------------------------")
print(" NDOT Concrete Mix Design – Weight Summary")
print("         (1 Cubic Yard of Concrete)")
print("---------------------------------------------")
print(f"Project Number:       {project_no}")
print(f"Class of Concrete:    {concrete_class}")
print("---------------------------------------------")
print(f"Cement (A):            {cement_A:8.1f} lb")
print(f"Fly Ash (B):           {fly_ash_B:8.1f} lb")
print(f"Silica Fume (C):       {silica_fume_C:8.1f} lb")
print(f"Other SCM (D):         {other_scm_D:8.1f} lb")
print("---------------------------------------------")
print(f"Fine Aggregate (Y):    {Y:8.0f} lb")
print(f"Coarse Aggregate (Z):  {Z:8.0f} lb")
print(f"Other Aggregate (AA):  {AA:8.0f} lb")
print("---------------------------------------------")
print(f"Water (Q):             {Q:8.0f} lb")
print("---------------------------------------------")
print("End of Mix Design Summary")

# Part 5 - Running Scenarios

For each scenario:
- Source citation (DOT doc / NDOT / engineering reference)
- Intended use case (e.g., pavement, bridge deck, cold weather)
- Input parameter set (must match your prompt model)

### Scenario 1: <Name>
- **Source:** <citation>
- **Use case:** <…>
- **Inputs:**
  - cement_content_lb_yd3 = …
  - wc_ratio = …
  - air_content_pct = …
  - sg_* = …
  - fine_agg_fraction = …
  - (<others>)


Suggested output table:

| Component | Weight (lb/yd³) | Notes |
|---|---:|---|
| Cement | <…> | |
| Water | <…> | |
| Fine Aggregate | <…> | |
| Coarse Aggregate | <…> | |
| Air (volumetric) | <…> | percent / ft³ |
| Total (sanity check) | <…> | |

Include:
- rounding rules (e.g., 1 decimal)
- clear units
- a short printed summary for a “client-readable” output

---

In [16]:
#Project Information for scenario
project_no = int(input("Enter project number: "))
class_of_concrete = input("Enter class of concrete: ")

Enter project number: 2
Enter class of concrete: 47B - 3500


In [ ]:
#Running Scenarios
#User can delete hashtags or enter user inputs to calulate for this class

#Source:
print("Source: Nebraska Department of Transportation. (2020). Standard specifications for highway construction (Section 1002: Portland cement concrete). http://www.nebraskatransportation.org/ref-man/specbooke/1000/1000-e20.pdf (nebraskatransportation.org in Bing)")

#Senario 1:
#For Class 47B - 3,500

#User can delete hashtags or enter user inputs to calulate for this class

#Cementitious materials
#cement_A = 470
#fly_ash_B = 120.0
#silica_fume_C = 0.0 
#other_scm_D = 0.0

#Water/cementitious ratio and air
#water_cement_ratio_E = 0.45 #max value
#air_content_F = 6 #within range given

#Aggregate perecnts
#percent_fine_G = 40.0 
#percent_coarse_H = 60.0 
#percent_other_I = 0.0

#Specific gravities 
#sg_cement_J = 3.15 
#sg_fly_ash_K = 2.40 
#sg_silica_fume_L = 2.20 
#sg_other_scm_M = 2.60 
#sg_fine_N = 2.65 
#sg_coarse_O = 2.65 
#sg_other_P = 2.65

#print("\n")
#print("Uses: pavement, curbs, gutter, sidewalks, and driveways")

#Calculations
Q = water_weight(cement_A, fly_ash_B, silica_fume_C, other_scm_D, water_cement_ratio_E)

R = vol_cement(cement_A, sg_cement_J)
S = fly_ash_vol(fly_ash_B, sg_fly_ash_K)
T = silica_vol(silica_fume_C, sg_silica_fume_L)
U = scm_vol(other_scm_D, sg_other_scm_M)

V = air_vol(air_content_F)
W = water_vol(Q)

X = agg_vol(R, S, T, U, V, W)

Y = fine_aggregate_Y(percent_fine_G, sg_fine_N, X)
Z = coarse_aggregate_Z(percent_coarse_H, sg_coarse_O, X)
AA = other_aggregate_AA(percent_other_I, sg_other_P, X)

#Results

#Table/Results
print("\n---------------------------------------------")
print(" NDOT Concrete Mix Design – Weight Summary")
print("         (1 Cubic Yard of Concrete)")
print("---------------------------------------------")
print(f"Project Number:        {project_no}")
print(f"Class of Concrete:     {class_of_concrete}")
print("---------------------------------------------")
print(f"Cement (A):            {cement_A:8.1f} lb")
print(f"Fly Ash (B):           {fly_ash_B:8.1f} lb")
print(f"Silica Fume (C):       {silica_fume_C:8.1f} lb")
print(f"Other SCM (D):         {other_scm_D:8.1f} lb")
print("---------------------------------------------")
print(f"Fine Aggregate (Y):    {Y:8.0f} lb")
print(f"Coarse Aggregate (Z):  {Z:8.0f} lb")
print(f"Other Aggregate (AA):  {AA:8.0f} lb")
print("---------------------------------------------")
print(f"Water (Q):             {Q:8.0f} lb")
print("---------------------------------------------")
print("End of Mix Design Summary")


In [ ]:
#Running Scenarios
#User can delete hashtags or enter user inputs to calulate for this class

#Source:
print("Source: City of Lincoln. (2020). Standard specifications for highway construction (Chapter 3: Portland Cement Concrete (PCC)). https://www.lincoln.ne.gov/files/sharedassets/public/v/2/ltu/transportation/standards/standard-specifications/2020/chapter3.pdf")

#Senario 1:
#For Class Flowable Fill

#User can delete hashtags or enter user inputs to calulate for this class

#Cementitious materials
#cement_A = 60
#fly_ash_B = 200
#silica_fume_C = 0.0 
#other_scm_D = 0.0

#Water/cementitious ratio and air
#water_cement_ratio_E = 0.45 #max value
#air_content_F = 10 

#Aggregate perecnts
#percent_fine_G = 100.0
#percent_coarse_H = 0.0
#percent_other_I = 0.0

#Specific gravities 
#sg_cement_J = 3.15 
#sg_fly_ash_K = 2.40 
#sg_silica_fume_L = 2.20 
#sg_other_scm_M = 2.60 
#sg_fine_N = 2.65 
#sg_coarse_O = 2.65 
#sg_other_P = 2.65

#print("\n")
#print("Uses: pipe bedding, utility backfill, and road construction")

#Calculations
Q = water_weight(cement_A, fly_ash_B, silica_fume_C, other_scm_D, water_cement_ratio_E)

R = vol_cement(cement_A, sg_cement_J)
S = fly_ash_vol(fly_ash_B, sg_fly_ash_K)
T = silica_vol(silica_fume_C, sg_silica_fume_L)
U = scm_vol(other_scm_D, sg_other_scm_M)

V = air_vol(air_content_F)
W = water_vol(Q)

X = agg_vol(R, S, T, U, V, W)

Y = fine_aggregate_Y(percent_fine_G, sg_fine_N, X)
Z = coarse_aggregate_Z(percent_coarse_H, sg_coarse_O, X)
AA = other_aggregate_AA(percent_other_I, sg_other_P, X)

#Results

#Table/Results
print("\n---------------------------------------------")
print(" NDOT Concrete Mix Design – Weight Summary")
print("         (1 Cubic Yard of Concrete)")
print("---------------------------------------------")
print(f"Project Number:        {project_no}")
print(f"Class of Concrete:     {class_of_concrete}")
print("---------------------------------------------")
print(f"Cement (A):            {cement_A:8.1f} lb")
print(f"Fly Ash (B):           {fly_ash_B:8.1f} lb")
print(f"Silica Fume (C):       {silica_fume_C:8.1f} lb")
print(f"Other SCM (D):         {other_scm_D:8.1f} lb")
print("---------------------------------------------")
print(f"Fine Aggregate (Y):    {Y:8.0f} lb")
print(f"Coarse Aggregate (Z):  {Z:8.0f} lb")
print(f"Other Aggregate (AA):  {AA:8.0f} lb")
print("---------------------------------------------")
print(f"Water (Q):             {Q:8.0f} lb")
print("---------------------------------------------")
print("End of Mix Design Summary")


In [ ]:
#Running Scenarios
#User can delete hashtags or enter user inputs to calulate for this class

#Source:
print("Source: City of Lincoln. (2020). Standard specifications for highway construction (Chapter 3: Portland Cement Concrete (PCC)). https://www.lincoln.ne.gov/files/sharedassets/public/v/2/ltu/transportation/standards/standard-specifications/2020/chapter3.pdf")

#Senario 1:
#For Class L-3500

#User can delete hashtags or enter user inputs to calulate for this class

#Cementitious materials
#cement_A = 564
#fly_ash_B = 0.0
#silica_fume_C = 0.0 
#other_scm_D = 0.0

#Water/cementitious ratio and air
#water_cement_ratio_E = 0.50 #Max Value
#air_content_F = 6 #within range given

#Aggregate perecnts
#percent_fine_G = 70.0
#percent_coarse_H = 30.0
#percent_other_I = 0.0

#Specific gravities 
#sg_cement_J = 3.15 
#sg_fly_ash_K = 2.40 
#sg_silica_fume_L = 2.20 
#sg_other_scm_M = 2.60 
#sg_fine_N = 2.65 
#sg_coarse_O = 2.65 
#sg_other_P = 2.65

#print("\n")
#print("Uses: pipe bedding, utility backfill, and road construction")

#Calculations
Q = water_weight(cement_A, fly_ash_B, silica_fume_C, other_scm_D, water_cement_ratio_E)

R = vol_cement(cement_A, sg_cement_J)
S = fly_ash_vol(fly_ash_B, sg_fly_ash_K)
T = silica_vol(silica_fume_C, sg_silica_fume_L)
U = scm_vol(other_scm_D, sg_other_scm_M)

V = air_vol(air_content_F)
W = water_vol(Q)

X = agg_vol(R, S, T, U, V, W)

Y = fine_aggregate_Y(percent_fine_G, sg_fine_N, X)
Z = coarse_aggregate_Z(percent_coarse_H, sg_coarse_O, X)
AA = other_aggregate_AA(percent_other_I, sg_other_P, X)

#Results

#Table/Results
print("\n---------------------------------------------")
print(" NDOT Concrete Mix Design – Weight Summary")
print("         (1 Cubic Yard of Concrete)")
print("---------------------------------------------")
print(f"Project Number:        {project_no}")
print(f"Class of Concrete:     {class_of_concrete}")
print("---------------------------------------------")
print(f"Cement (A):            {cement_A:8.1f} lb")
print(f"Fly Ash (B):           {fly_ash_B:8.1f} lb")
print(f"Silica Fume (C):       {silica_fume_C:8.1f} lb")
print(f"Other SCM (D):         {other_scm_D:8.1f} lb")
print("---------------------------------------------")
print(f"Fine Aggregate (Y):    {Y:8.0f} lb")
print(f"Coarse Aggregate (Z):  {Z:8.0f} lb")
print(f"Other Aggregate (AA):  {AA:8.0f} lb")
print("---------------------------------------------")
print(f"Water (Q):             {Q:8.0f} lb")
print("---------------------------------------------")
print("End of Mix Design Summary")


In [ ]:
#Running Scenarios
#User can delete hashtags or enter user inputs to calulate for this class

#Source:
print("Source: City of Lincoln. (2020). Standard specifications for highway construction (Chapter 3: Portland Cement Concrete (PCC)). https://www.lincoln.ne.gov/files/sharedassets/public/v/2/ltu/transportation/standards/standard-specifications/2020/chapter3.pdf")

#Senario 1:
#For Class LB-2750

#User can delete hashtags or enter user inputs to calulate for this class

#Cementitious materials
#cement_A = 423
#fly_ash_B = 0.0
#silica_fume_C = 0.0 
#other_scm_D = 0.0

#Water/cementitious ratio and air
#water_cement_ratio_E = 0.60 #max value
#air_content_F = 6 #within range given

#Aggregate perecnts
#percent_fine_G = 60.0
#percent_coarse_H = 40.0
#percent_other_I = 0.0

#Specific gravities 
#sg_cement_J = 3.15 
#sg_fly_ash_K = 2.40 
#sg_silica_fume_L = 2.20 
#sg_other_scm_M = 2.60 
#sg_fine_N = 2.65 
#sg_coarse_O = 2.65 
#sg_other_P = 2.65

#print("\n")
#print("Uses: pipe bedding, utility backfill, and road construction")

#Calculations
Q = water_weight(cement_A, fly_ash_B, silica_fume_C, other_scm_D, water_cement_ratio_E)

R = vol_cement(cement_A, sg_cement_J)
S = fly_ash_vol(fly_ash_B, sg_fly_ash_K)
T = silica_vol(silica_fume_C, sg_silica_fume_L)
U = scm_vol(other_scm_D, sg_other_scm_M)

V = air_vol(air_content_F)
W = water_vol(Q)

X = agg_vol(R, S, T, U, V, W)

Y = fine_aggregate_Y(percent_fine_G, sg_fine_N, X)
Z = coarse_aggregate_Z(percent_coarse_H, sg_coarse_O, X)
AA = other_aggregate_AA(percent_other_I, sg_other_P, X)

#Results

#Table/Results
print("\n---------------------------------------------")
print(" NDOT Concrete Mix Design – Weight Summary")
print("         (1 Cubic Yard of Concrete)")
print("---------------------------------------------")
print(f"Project Number:        {project_no}")
print(f"Class of Concrete:     {class_of_concrete}")
print("---------------------------------------------")
print(f"Cement (A):            {cement_A:8.1f} lb")
print(f"Fly Ash (B):           {fly_ash_B:8.1f} lb")
print(f"Silica Fume (C):       {silica_fume_C:8.1f} lb")
print(f"Other SCM (D):         {other_scm_D:8.1f} lb")
print("---------------------------------------------")
print(f"Fine Aggregate (Y):    {Y:8.0f} lb")
print(f"Coarse Aggregate (Z):  {Z:8.0f} lb")
print(f"Other Aggregate (AA):  {AA:8.0f} lb")
print("---------------------------------------------")
print(f"Water (Q):             {Q:8.0f} lb")
print("---------------------------------------------")
print("End of Mix Design Summary")
